In [140]:
import tiktoken
import torch
import os
import re
import urllib.request
import numpy as np

## Tokenization

In [141]:
with open("the-verdict.txt", 'r') as f:
    raw_text = f.read()
print(f"Total number of characters : {len(raw_text)}")
print(raw_text[:99])

Total number of characters : 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [142]:
preprocessed = re.split('([,.:;?_!"()\']|--|\s)', raw)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

In [143]:
vocab_size = len(all_tokens)

vocab = {token:integer for integer,token in enumerate(all_tokens)}

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [144]:
class SimpleTokenizer_V1:

    def __init__(self,vocab):

        self.str_to_int  = vocab
        self.int_to_str  = {i:s for s,i in vocab.items()}

    def encode(self,text):

        preprocessed = re.split('([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>" for item in preprocessed
            ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):

        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [145]:
token = SimpleTokenizer_V1(vocab)
token.decode(token.encode("I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no Vishal"))

'I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was no <|unk|>'

In [146]:
# Using BPE Tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

tokenizer.decode(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

## Data Sampling

In [147]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [148]:
from torch.utils.data import Dataset,DataLoader

class GPTDataset(Dataset):
    def __init__(self,text,tokenizer, context_size, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0,len(token_ids)-context_size, stride):
            self.input_ids.append(torch.tensor(token_ids[i:i+context_size]))
            self.target_ids.append(torch.tensor(token_ids[i+1:i+context_size+1]))

    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [149]:
def create_dataloader(text, 
                      context_size = 256, 
                      stride = 128, 
                      batch_size = 4, 
                      shuffle = True, 
                      drop_last = True, 
                      num_workers = 0):
    
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDataset(text = text, tokenizer= tokenizer, context_size= context_size, stride = stride)

    dataloader = DataLoader(
        dataset=dataset,
        shuffle=shuffle,
        batch_size=batch_size,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [150]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(
    raw_text, batch_size=1, context_size=4, stride=1, shuffle=False
)

data = iter(dataloader)
input_ids, target_ids = next(data)
print("Input IDs :\n", input_ids)
print("\nTarget IDs :\n", target_ids)

Input IDs :
 tensor([[  40,  367, 2885, 1464]])

Target IDs :
 tensor([[ 367, 2885, 1464, 1807]])


## Token Embeddings

In [167]:
vocab_size = 50257 # BPE Tokenizer has 50257 vocabs
embedding_dim = 256
context_size = 4
position = torch.arange(context_size)
torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)
positional_encode_layer = torch.nn.Embedding(context_size, embedding_dim)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(
    raw_text, batch_size=8, context_size=4, stride=4, shuffle=False
)

data = iter(dataloader)
input_ids, target_ids = next(data)
print(input_ids)

token_position_embed = embedding_layer(input_ids) + positional_encode_layer(position)
token_position_embed.shape

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


torch.Size([8, 4, 256])